In [1]:
!pip install langchain-groq
!pip install langchain-community
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42

In [2]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import gdown
import json

In [3]:
os.environ["GROQ_API_KEY"] = "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"
model = ChatGroq(temperature=0.8, model="llama3-8b-8192")

In [4]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 5.2 MB/s eta 0:00:00


In [6]:
pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 5.2 MB/s eta 0:00:00


In [8]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 59.6 MB/s eta 0:00:00


In [9]:
# def load_pdf_text(pdf_path):
#     loader = PyPDFLoader(pdf_path)
#     pages = loader.load_and_split()
#     return [page.page_content for page in pages]
# pdf_path = "/content/BDM Capstone Project Document for 2024.pdf"
# document_texts = load_pdf_text(pdf_path)
# embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# vector_store = FAISS.from_texts(document_texts, embedder)

def load_multiple_pdfs(pdf_paths):
    all_texts = []
    for pdf_path in pdf_paths:
        loader = PyPDFLoader(pdf_path)
        pages = loader.load_and_split()
        all_texts.extend([page.page_content for page in pages])
    return all_texts

pdf_paths = ["/content/SoP.pdf", "/content/Few_Problems.pdf", "/content/BDM Capstone Project Document for 2024.pdf"]
document_texts = load_multiple_pdfs(pdf_paths)
embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_texts(document_texts, embedder)

In [10]:
retrieval_chain = ConversationalRetrievalChain.from_llm(model, retriever=vector_store.as_retriever())

In [11]:
print("Chatbot is ready! Ask questions about the PDF content, or type 'stop' to exit.")
chat_history = []

while True:
    user_input = input("You: ")
    if user_input.lower() == 'stop':
        print("Chatbot: Goodbye!")
        break
    response = retrieval_chain.invoke({"question": user_input, "chat_history": chat_history})
    chat_history.append((user_input, response["answer"]))
    answer = response["answer"]
    formatted_answer = "\n".join(" ".join(answer.split()[i:i+15]) for i in range(0, len(answer.split()), 15))
    print("Chatbot: ", formatted_answer)

Chatbot is ready! Ask questions about the PDF content, or type 'stop' to exit.
You: hello
Chatbot:  Hello! It seems you'd like to discuss a business development management (BDM) capstone project. I'd
be happy to help you with that. Could you please provide more context or specify
what you'd like to know or discuss?
You: stop
Chatbot: Goodbye!
